In [ ]:
from load_data import pull_from_postgres
from psycopg2.sql import SQL
import pandas as pd

# pull covid test results and test types from postgres
test_data = pull_from_postgres(SQL("""
    SELECT
        features.f10 test_result, features.test_week_start, features.user_id, answers.element test_type
    FROM
        datenspende_derivatives.homogenized_features features, datenspende.answers answers
    WHERE
        features.test_week_start >= '2021-10-01' AND features.questionnaire_session = answers.questionnaire_session AND answers.question = 91
    """)).replace({'test_type': {547: 'PCR', 548: 'Antigen', 549: 'Antibody', 550: 'Unknown'}})

# convert date to pandas datetime
test_data['date'] = pd.to_datetime(test_data['test_week_start'])
test_data.drop(columns=['test_week_start'], inplace=True)
len(test_data['user_id'].unique())

In [ ]:
# count missing tests as negative
res1 = test_data.replace(
    {True: 1, False: 0, None: 0, float("nan"): 0}
).reset_index()[["test_result", "date", "user_id", 'test_type']]

# count only first positive test to avoid counting active infections again.
# TODO: consider reinfections
only_first_positive_tests = (
    res1[res1["test_result"] == 1]
    .reset_index()
    .sort_values(["user_id", "date"])
    .groupby(["user_id"])
    .agg({"test_result": "first", 'test_type': "first", "date": lambda x: list(x)[0]})
    .reset_index()
)
all_negative_tests = res1[res1["test_result"] == 0].reset_index()

infections = pd.concat(
    [only_first_positive_tests, all_negative_tests]
).drop(columns=['index'])

infections.sort_values('date')

In [ ]:
# aggregate by user_id
df = infections.groupby(['test_type', 'date']).agg({'test_result': ['sum', 'count']}).reset_index().sort_values(['date', 'test_type'])
df.columns = ['test_type', 'date', 'sum', 'count']

# count all reports (with or without test) for a given day
all_tests = df.groupby(['date']).agg({'sum': 'sum', 'count': 'sum'}).reset_index()
all_tests['test_type'] = 'all'
sum_of_daily_tests = pd.concat([df, all_tests]).reset_index(drop=True).sort_values(['date', 'test_type'])
sum_of_daily_tests

In [ ]:
# generate dataframe with sum of positive tests per type per day
positive_tests = sum_of_daily_tests[['test_type', 'sum', 'date']].rename(columns={'sum': 'positive_tests'}).set_index(['date', 'test_type']).unstack('test_type')
positive_tests.columns = positive_tests.columns.droplevel(0)

# generate dataframe with sum of all reports per day
all_daily_reports = sum_of_daily_tests[['test_type', 'count', 'date']].rename(columns={'count': 'all_daily_reports'}).set_index(['date', 'test_type']).unstack('test_type')
all_daily_reports.columns = all_daily_reports.columns.droplevel(0)
all_daily_reports = all_daily_reports[['all']].rename(columns={'all': 'all_daily_reports'})

daily_aggregate_test_data = positive_tests.merge(all_daily_reports, on='date', how='outer')
daily_aggregate_test_data

In [ ]:
import matplotlib.pyplot as plt

all_daily_reports = sum_of_daily_tests[['test_type', 'count', 'date']].rename(columns={'count': 'all_daily_reports'}).set_index(['date', 'test_type']).unstack('test_type')
all_daily_reports.columns = all_daily_reports.columns.droplevel(0)

# normalize by all daily reports
for column in all_daily_reports.columns:
    all_daily_reports[column] = all_daily_reports[column] / all_daily_reports['all']

fig, ax = plt.subplots(figsize=(10, 6))

# plot relative share of test types among all reported tests per day
all_daily_reports[['Antibody', 'Antigen', 'PCR', 'Unknown']].plot.area(ax=ax)
ax.set_ylabel('Relative share of test types')
ax.set_xlabel('Date')
ax.set_title('Relative share of test types among all reported tests per day')
ax.set_xlim(pd.to_datetime('2021-10-01'), pd.to_datetime('2022-07-2'))
ax.set_ylim(0, 1)

all_daily_reports.to_csv('all_daily_reports.csv')

In [ ]:
import matplotlib.pyplot as plt

df = daily_aggregate_test_data.copy()

# divide by all
for column in df.columns:
    df[column] = df[column] / df['all']

df.dropna(subset=['all'], inplace=True)

# take 7 day rolling mean
df = df.rolling(7).mean()

fig, ax = plt.subplots(figsize=(10, 5))

# plot relative share of antibody, antigen, PCR, unknown tests
df[['Antibody', 'Antigen', 'PCR', 'Unknown']].plot.area(ax=ax)
ax.set_xlabel('Date')
ax.set_ylabel('Relative share of tests')
ax.legend(loc='upper left')
ax.set_title('Relative share of test types among positive tests per day')
ax.set_xlim(pd.to_datetime('2021-10-20'), pd.to_datetime('2022-07-2'))
ax.set_ylim(0, 1)
ax.text(0.92, 0.95, 'A)', transform=ax.transAxes, fontsize=24, verticalalignment='top')
fig.savefig('relative_shares_of_test_type_among_positive_tests_per_day.png')

df.to_csv('relative_shares_of_test_type_among_positive_tests_per_day.csv')

In [ ]:
# plot test positivity rate for each test type per day
sum_of_daily_tests['rate'] = sum_of_daily_tests['sum'] / sum_of_daily_tests['count']

df = sum_of_daily_tests[['rate', 'date', 'test_type']].set_index(['date', 'test_type']).unstack('test_type')
df.columns = df.columns.droplevel(0)
df = df.rolling(7).mean()

fig, ax = plt.subplots(figsize=(10, 8))
df[['PCR', 'Antigen', 'all']].plot(ax=ax)

ax.set_xlabel('Date')
ax.set_ylabel('Test positivity rate')
ax.legend(loc='upper left')
ax.set_title('Test positivity rate per day per test type')
plt.show()

In [ ]:
# calcualte 7 day incidence per test type

df = daily_aggregate_test_data.copy()
incidence = pd.DataFrame()
for column in df.columns:
    incidence[column + ' incidence'] = df[column] / df['all_daily_reports'] * 100_000

incidence = incidence[['Antigen incidence', 'PCR incidence', 'all incidence']].dropna().rolling('7D').mean()

fig, ax = plt.subplots(figsize=(10, 8))
incidence.plot(ax=ax)

In [ ]:
incidence_official = pull_from_postgres(SQL(
    """
    SELECT date_of_report date, incidence_7d_per_100k official_incidence FROM coronacases.german_counties_incidence WHERE location_level = 0;
    """))
incidence_official

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

columns = incidence.columns.drop('all incidence')

df = incidence_official.merge(incidence, on='date').sort_values(by="date").reset_index(drop=True).query('date>20220101').drop(columns=['all incidence'])

fig, ax = plt.subplots(figsize=(12, 6))
axb = ax.twinx()

ax1 = df.plot(x="date", y="official_incidence", ax=ax, label="incidence as officially reported", color='blue', legend=False)
ax1 = df.plot(x="date", y="official_incidence", ax=ax, label="incidence as officially reported", color='blue', legend=False)

colors = ['red', 'green', 'black']
incidence_lines = []
for column, color in zip(columns, colors):
    incidence_lines.append(Line2D([0], [0], color=color, label=column))
    ax2 = df.plot(x="date", y=column, ax=axb, label="incidence from surveys", color=color, legend=False)

# create legend manually
labels = ['incidence as officially reported', *incidence.columns.values]
blue_line = Line2D([0], [0], color='blue', label='official incidence')
red_line = Line2D([0], [0], color='red', label='self reported incidence')
ax.legend([blue_line, *incidence_lines], labels, loc='upper left')

# label axes
ax.set_xlabel('Date')
ax.set_ylabel('Incidence calculated from surveys as 7 day average per 100000')
axb.set_ylabel('Incidence as officially reported')

# label plot as B)
ax.text(0.92, 0.95, 'B)', transform=ax.transAxes, fontsize=24, verticalalignment='top')

fig.savefig('incidence_pcr_vs_antigen_vs_reported.png')

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

columns = incidence.columns

df = incidence_official.merge(incidence, on='date').sort_values(by="date").reset_index(drop=True).query('date>20220101')

fig, ax = plt.subplots(figsize=(12, 6))
axb = ax.twinx()

ax1 = df.plot(x="date", y="official_incidence", ax=ax, label="incidence as officially reported", color='blue', legend=False)
ax1 = df.plot(x="date", y="official_incidence", ax=ax, label="incidence as officially reported", color='blue', legend=False)

colors = ['red', 'green', 'black']
incidence_lines = []
for column, color in zip(columns, colors):
    incidence_lines.append(Line2D([0], [0], color=color, label=column))
    ax2 = df.plot(x="date", y=column, ax=axb, label="incidence from surveys", color=color, legend=False)

# create legend manually
labels = ['incidence as officially reported', *incidence.columns.values]
blue_line = Line2D([0], [0], color='blue', label='official incidence')
red_line = Line2D([0], [0], color='red', label='self reported incidence')
ax.legend([blue_line, *incidence_lines], labels, loc='upper left')

# label axes
ax.set_xlabel('Date')
ax.set_ylabel('Incidence calculated from surveys as 7 day average per 100000')
axb.set_ylabel('Incidence as officially reported')


ax.text(0.9, 0.95, 'C)', transform=ax.transAxes, fontsize=24, verticalalignment='top')

fig.savefig('incidence_all.png')

In [ ]:
columns = incidence.columns.drop('all incidence')

df = incidence_official.merge(incidence, on='date').sort_values(by="date").reset_index(drop=True).query('date>20220101').drop(columns=['all incidence'])

fig, ax = plt.subplots(figsize=(12, 6))

ax1 = df.plot(x="date", y="official_incidence", ax=ax, label="incidence as officially reported", color='blue', legend=False)

colors = ['red', 'green', 'black']
incidence_lines = []
for column, color in zip(columns, colors):
    incidence_lines.append(Line2D([0], [0], color=color, label=column))
    ax2 = df.plot(x="date", y=column, ax=ax, label="incidence from surveys", color=color, legend=False)

# create legend manually
labels = ['incidence as officially reported', *incidence.columns.values]
blue_line = Line2D([0], [0], color='blue', label='official incidence')
red_line = Line2D([0], [0], color='red', label='self reported incidence')
ax.legend([blue_line, *incidence_lines], labels, loc='upper left')

# label axes
ax.set_xlabel('Date')
ax.set_ylabel('Incidence as 7 day average per 100000')
axb.set_ylabel('Incidence as officially reported')

ax.text(0.92, 0.95, 'D)', transform=ax.transAxes, fontsize=24, verticalalignment='top')

fig.savefig('incidence_pcr_vs_antigen_vs_reported_on_same_ax.png')

In [ ]:
df

In [ ]:
df.to_csv('incidence_pcr_vs_antigen_vs_reported.csv')